# Deploy a Hyper-Segmented Model with TDStone2 in less than a minute
# Using Scikit Learn Regressor Pipeline

## 1 - Connect to Vantage

In [1]:
import teradataml as tdml
import json
import warnings
warnings.filterwarnings('ignore')
tdml.__version__

'20.00.00.03'

In [2]:
with open('../connections/me.json','r') as f:
    Param_ = json.load(f)

with open('../connections/csae2.json','r') as f:
    Param = json.load(f)

with open('../connections/csae.json','r') as f:
    Param_ = json.load(f)

with open('../connections/vantage24.json','r') as f:
    Param_ = json.load(f)

tdml.create_context(**Param)

Engine(teradatasql://:***@sto-xo0saiv0ot1sxmj2.env.clearscape.teradata.com?DATABASE=demo_user&user=DEMO_USER)

## 2 - The Hyper-Segmented Dataset

In [3]:
dataset = tdml.DataFrame(tdml.in_schema(Param['database'],'dataset_00'))
dataset

Partition_ID,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,flag,Y1,Y2,FOLD
6,30010,-0.6159002164442977,0.14080392983716347,0.15889642964183437,1.085758574614208,1.0241507354618067,2.2879807373865955,1.248066060664809,-0.2606740117351424,-0.7335729690489351,1,-3.5550050814907093,0,train
6,30013,2.014246542658942,1.1276736739000246,0.9270218987818085,-1.481457917950641,-0.250524210252159,1.3736025167780217,0.11833821335213064,0.20324762078742337,-0.2578441088485678,1,0.07612901906702242,1,train
6,30014,-0.319454244703348,-0.26647934999098355,-0.8855616476232634,1.0956373292993418,0.4434036622230671,-0.7549072657852008,-0.3220580895497687,0.3554228691175283,0.37396912918459113,1,2.1522057679593285,1,train
6,30015,-0.5504866066204547,-1.574888997452571,-0.77900638523968,-1.483422934695842,0.7921706663712172,0.9782544528314772,1.8178991885389517,-0.6817641511315384,-2.0605602446759614,1,-1.3233448815023607,0,train
6,30018,0.044239193425873395,-1.6933428540319742,-0.056453960392806084,1.2965356620186717,-0.21712514958245874,-0.6307579561491885,-1.7585648287677427,1.7064927180056626,-0.1277308494333046,0,-1.5730006932435168,0,train
6,30019,-0.8104370397714358,-0.8973745738715776,0.14048511415936155,0.017814656048642106,0.12192930944603292,0.0899094795998071,0.5012314471422181,0.14968813866691935,0.7221007636114213,1,-1.1312422594196083,0,train
6,30017,-1.2254194266972793,0.22362867806228284,1.6718585615517347,0.8667485952008627,-1.233638698141816,-0.5351216076408845,-0.17009354142100885,1.2487484730759668,0.507666715822585,0,-0.9344959332955182,0,train
6,30011,0.9085979319083528,-0.12273633906514257,-0.1924983184913712,-0.08551225690095689,-0.529210304049312,-1.424598154291598,-0.7672687747877786,-1.23778383521349,-0.22277129947493943,0,2.4392101587122745,1,train
6,30009,0.9896646792044919,0.4844067783112346,-0.828941349703854,-0.08840387865308055,-0.6517543675902613,-0.5395411066877841,2.664306600799727,-2.3323792258512976,-1.5727275532456393,0,-0.04463101382350978,0,train
6,30006,1.1189112609154879,-0.1394489429107103,0.3675636706503788,1.2139417083538135,0.659723049081594,0.42289915056236166,0.9614196487091969,-1.8089968924242852,-0.07330263194893605,0,1.7074154119360592,1,train


In [4]:
summary = dataset.groupby('Partition_ID').count()
summary

Partition_ID,count_ID,count_X1,count_X2,count_X3,count_X4,count_X5,count_X6,count_X7,count_X8,count_X9,count_flag,count_Y1,count_Y2,count_FOLD
4,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
9,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
1,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
6,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000


In [5]:
summary.shape

(4, 15)

## 3 - Hyper-segmented model deployment

### 3.1 - Engineering of the scikit-learn classifier pipeline

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoLarsCV
from sklearn.pipeline import Pipeline

# Setting up the pipeline for regression with LassoLarsCV
steps_lasso_lars_cv = [
    ('scaler', StandardScaler()),
    ('lasso_lars_cv', LassoLarsCV(
        cv        = 5,     # Number of cross-validation folds. Adjust this based on your dataset size and requirements.
        max_iter  = 1000,  # Maximum number of iterations. Adjust as needed.
    ))
]

### 3.2 - Deployment of the scikit-learn pipeline

In [7]:
from tdstone2.tdshypermodel import HyperModel
from tdstone2.tdstone import TDStone
sto = TDStone(schema_name = Param['database'], SEARCHUIFDBPATH = Param['user'])

In [ ]:
model_parameters = {
    "target": 'Y1',
    "column_categorical": ['flag'],
    "column_names_X": ['X1','X2','X3','X4','X5','X6','X7','X8','X9','flag']
}

In [9]:
%%time
model = HyperModel(tdstone            = sto,
                   metadata           = {'project': 'test'},
                   skl_pipeline_steps = steps_lasso_lars_cv,
                   model_parameters   = model_parameters,
                   dataset            = tdml.in_schema(Param['database'],'dataset_00'),
                   id_row             = 'ID',
                   id_partition       = 'Partition_ID',
                   id_fold            = 'FOLD',
                   fold_training      = 'train')

registration of mapper with id = 2e06e335-a352-4480-805d-182f66dca5e7
creation of dedicated mapper table : demo_user.TDS_MAPPER_2e06e335_a352_4480_805d_182f66dca5e7
creation of the on clause view demo_user.TDS_ON_CLAUSE_TRAINING_2e06e335_a352_4480_805d_182f66dca5e7
creation of the sto view demo_user.TDS_STO_TRAINING_2e06e335_a352_4480_805d_182f66dca5e7
registration of mapper with id = cddb9b47-acc7-4638-a61b-802aea11f0d7
creation of dedicated mapper table : demo_user.TDS_MAPPER_cddb9b47_acc7_4638_a61b_802aea11f0d7
creation of the on clause view demo_user.TDS_ON_CLAUSE_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7
creation of the volatile table on on clause view demo_user.TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7
creation of the sto view demo_user.TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7
register hyper model with id : f17adbc1-cd51-4281-87ca-2fb179d4ccd0
hyper model : f17adbc1-cd51-4281-87ca-2fb179d4ccd0
CPU times: total: 141 ms
Wall time: 28.2 s


In [10]:
sto.list_hyper_models()

CREATION_DATE,ID,ID_MODEL,ID_MAPPER_TRAINING,ID_MAPPER_SCORING,METADATA
2025-01-08 05:26:48.160000-05:,654b028b-741e-4a46-8e5d-d966ac8a07b1,1feb2a86-622c-4c56-8e94-4c367d1a232e,05842693-ddf1-4589-8d7b-a7ea4e952b95,a93604aa-2204-45a6-b4f5-1edab0414da1,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 05:31:04.970000-05:,c5ef0032-dd20-4b48-817e-6fe7e3c42e55,505ea202-bbf4-4b2c-9639-2a18176d97f1,dc1cdd3e-111f-440e-ae0e-c0b3336c0716,431b1e0c-57a4-4ae8-bb3e-3a2372349857,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 06:33:41.570000-05:,f17adbc1-cd51-4281-87ca-2fb179d4ccd0,479b48a6-e0e5-4d18-b637-de6f1ee63d8a,2e06e335-a352-4480-805d-182f66dca5e7,cddb9b47-acc7-4638-a61b-802aea11f0d7,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 05:36:52.330000-05:,91fb8620-0b80-4cec-a8a5-e1de531e608e,06e9bc9e-32f1-4f7b-88f9-5a09e1ac8204,72ef3ebe-73d5-4a13-89d3-e093b53c81d1,d14273b9-df63-444b-98ce-0bad17935b12,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 06:28:41.130000-05:,533fd2f6-cdf9-4c18-bfd1-11f6c017a587,0fe42030-d412-411b-9695-8571c07507a0,fca98df7-3e26-41f5-a8a8-0036c348fb9a,57d6b26b-7691-4de9-b465-1d48affe93a8,"{""user"": ""dm250067"", ""project"": ""test""}"


In [11]:
id = sto.list_hyper_models()[['ID','CREATION_DATE']].to_pandas().reset_index().sort_values('CREATION_DATE', ascending=False).iloc[0,0]
id

'f17adbc1-cd51-4281-87ca-2fb179d4ccd0'

## 4 - Execution of the deployed hypermodel

### 4.1 - Models Training

In [12]:
%%time
model.train()

insert trained models in demo_user.TDS_TRAINED_MODELS_2e06e335_a352_4480_805d_182f66dca5e7
- access pickle models in demo_user.V_TDS_TRAINED_MODELS_2e06e335_a352_4480_805d_182f66dca5e7_PICKLE
- access  onnx  models in demo_user.V_TDS_TRAINED_MODELS_2e06e335_a352_4480_805d_182f66dca5e7_ONNX
- onnx byom catalog    in demo_user.V_TDS_TRAINED_MODELS_2e06e335_a352_4480_805d_182f66dca5e7_BYOM_CATALOG
CPU times: total: 31.2 ms
Wall time: 9.75 s


In [13]:
model.get_trained_models().groupby('TD_TIMECODE').count()

demo_user TDS_MODEL_REPOSITORY


TD_TIMECODE,count_ID_PROCESS,count_Partition_ID,count_ID_MODEL,count_ID_TRAINED_MODEL,count_MODEL_TYPE,count_STATUS
9999-01-01 00:00:00.000000-04:,4,4,4,4,4,4


### 4.2 - Model Scoring

In [14]:
%%time
model.score()

creation of the on clause view demo_user.TDS_ON_CLAUSE_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7
creation of the volatile table on on clause view demo_user.TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7
insert scores in demo_user.TDS_SCORES_cddb9b47_acc7_4638_a61b_802aea11f0d7
CPU times: total: 31.2 ms
Wall time: 19.9 s


In [15]:
model.get_model_predictions().groupby('TD_TIMECODE').count()

demo_user TDS_SCORES_cddb9b47_acc7_4638_a61b_802aea11f0d7


TD_TIMECODE,count_ID_PROCESS,count_ID_TRAINED_MODEL,count_Partition_ID,count_ID,count_Y1_prediction
9999-01-01 00:00:00.000000-04:,40000,40000,40000,40000,40000


In [16]:
model.get_model_predictions()

demo_user TDS_SCORES_cddb9b47_acc7_4638_a61b_802aea11f0d7


TD_TIMECODE,ID_PROCESS,ID_TRAINED_MODEL,Partition_ID,ID,Y1_prediction
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,0fd9d643-9a31-4560-9ab9-4e8fbc7194cb,6,39884,0.0709809863307359
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,21115,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,0ea9f0ea-01ae-47b5-a207-a75487efe313,9,18864,-0.05335791349929137
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,23179,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,29371,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,20083,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,27307,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,0ea9f0ea-01ae-47b5-a207-a75487efe313,9,13198,-0.04490876728584044
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,26275,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,22147,0.018697822801857347


## 5 - A bit of administration: Model Lineage

### 5.1 - Access to the list of deployed codes

In [17]:
sto.list_codes()

ID,CODE_TYPE,METADATA
48dea5f2-7862-4f1a-9b71-3f03661772e2,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"
85be8eae-22c2-4a82-9ec1-a2af5c134374,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"
0a923c33-9369-407e-80a0-187ffa2a9837,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"
c88da75f-4c35-4fb4-8b2b-52105610adf7,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"
2498d459-7d97-46c3-8cec-40f350c7230f,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"


### 5.2 - List of deployed models (code + parameters)

In [18]:
sto.list_models()

ID,ID_CODE,ARGUMENTS,METADATA
505ea202-bbf4-4b2c-9639-2a18176d97f1,0a923c33-9369-407e-80a0-187ffa2a9837,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""pickle""]}, ""model_parameters"": {""target"": ""Y1"", ""column_categorical"": [""flag""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""arguments"": {""regressor__max_depth"": 5, ""regressor__n_estimators"": 95}}}","{""user"": ""dm250067"", ""project"": ""test""}"
06e9bc9e-32f1-4f7b-88f9-5a09e1ac8204,48dea5f2-7862-4f1a-9b71-3f03661772e2,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""pickle""]}, ""model_parameters"": {""column_categorical"": [""flag""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""target"": """", ""arguments"": {""one_class_svm__gamma"": ""auto"", ""one_class_svm__nu"": 0.05}}}","{""user"": ""dm250067"", ""project"": ""test""}"
479b48a6-e0e5-4d18-b637-de6f1ee63d8a,85be8eae-22c2-4a82-9ec1-a2af5c134374,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""pickle""]}, ""model_parameters"": {""target"": ""Y1"", ""column_categorical"": [""flag""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""arguments"": {""lasso_lars_cv__cv"": 5, ""lasso_lars_cv__max_iter"": 1000}}}","{""user"": ""dm250067"", ""project"": ""test""}"
1feb2a86-622c-4c56-8e94-4c367d1a232e,c88da75f-4c35-4fb4-8b2b-52105610adf7,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""pickle""]}, ""model_parameters"": {""target"": ""Y2"", ""column_categorical"": [""flag"", ""Y2""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""arguments"": {""classifier__max_depth"": 5, ""classifier__n_estimators"": 95}}}","{""user"": ""dm250067"", ""project"": ""test""}"
0fe42030-d412-411b-9695-8571c07507a0,2498d459-7d97-46c3-8cec-40f350c7230f,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""onnx"", ""pickle""]}, ""model_parameters"": {""column_categorical"": [""flag""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""target"": """", ""arguments"": {""one_class_svm__gamma"": ""auto"", ""one_class_svm__nu"": 0.05}}}","{""user"

### 5.3 - List of available mappers (mapping between partitions and models or trained models)

In [19]:
sto.list_mappers()

ID,MAPPER_TYPE,TABLE_NAME,CODE_REPOSITORY,MODEL_REPOSITORY,TRAINED_MODEL_REPOSITORY,FEATURE_REPOSITORY,SCORES_REPOSITORY,DATASET_OBJECT,COL_ID_ROW,COL_ID_PARTITION,COL_FOLD,ON_CLAUSE_VIEW,STO_VIEW,METADATA
a93604aa-2204-45a6-b4f5-1edab0414da1,scoring,TDS_MAPPER_a93604aa_2204_45a6_b4f5_1edab0414da1,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_05842693_ddf1_4589_8d7b_a7ea4e952b95,None,TDS_SCORES_a93604aa_2204_45a6_b4f5_1edab0414da1,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_SCORING_a93604aa_2204_45a6_b4f5_1edab0414da1,TDS_STO_SCORING_a93604aa_2204_45a6_b4f5_1edab0414da1,"{""user"": ""dm250067""}"
05842693-ddf1-4589-8d7b-a7ea4e952b95,training,TDS_MAPPER_05842693_ddf1_4589_8d7b_a7ea4e952b95,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_05842693_ddf1_4589_8d7b_a7ea4e952b95,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_05842693_ddf1_4589_8d7b_a7ea4e952b95,TDS_STO_TRAINING_05842693_ddf1_4589_8d7b_a7ea4e952b95,"{""user"": ""dm250067""}"
d14273b9-df63-444b-98ce-0bad17935b12,scoring,TDS_MAPPER_d14273b9_df63_444b_98ce_0bad17935b12,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_72ef3ebe_73d5_4a13_89d3_e093b53c81d1,None,TDS_SCORES_d14273b9_df63_444b_98ce_0bad17935b12,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_SCORING_d14273b9_df63_444b_98ce_0bad17935b12,TDS_STO_SCORING_d14273b9_df63_444b_98ce_0bad17935b12,"{""user"": ""dm250067""}"
2e06e335-a352-4480-805d-182f66dca5e7,training,TDS_MAPPER_2e06e335_a352_4480_805d_182f66dca5e7,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_2e06e335_a352_4480_805d_182f66dca5e7,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_2e06e335_a352_4480_805d_182f66dca5e7,TDS_STO_TRAINING_2e06e335_a352_4480_805d_182f66dca5e7,"{""user"": ""dm250067""}"
431b1e0c-57a4-4ae8-bb3e-3a2372349857,scoring,TDS_MAPPER_431b1e0c_57a4_4ae8_bb3e_3a2372349857,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,None,TDS_SCORES_431b1e0c_57a4_4ae8_bb3e_3a2372349857,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,"{""user"": ""dm250067""}"
cddb9b47-acc7-4638-a61b-802aea11f0d7,scoring,TDS_MAPPER_cddb9b47_acc7_4638_a61b_802aea11f0d7,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_2e06e335_a352_4480_805d_182f66dca5e7,None,TDS_SCORES_cddb9b47_acc7_4638_a61b_802aea11f0d7,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,"{""user"": ""dm250067""}"
fca98df7-3e26-41f5-a8a8-0036c348fb9a,training,TDS_MAPPER_fca98df7_3e26_41f5_a8a8_0036c348fb9a,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_fca98df7_3e26_41f5_a8a8_0036c348fb9a,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_fca98df7_3e26_41f5_a8a8_0036c348fb9a,TDS_STO_TRAINING_fca98df7_3e26_41f5_a8a8_0036c348fb9a,"{""user"": ""dm250067""}"
72ef3ebe-73d5-4a13-89d3-e093b53c81d1,training,TDS_MAPPER_72ef3ebe_73d5_4a13_89d3_e093b53c81d1,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_72ef3ebe_73d5_4a13_89d3_e093b53c81d1,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_72ef3ebe_73d5_4a13_89d3_e093b53c81d1,TDS_STO_TRAINING_72ef3ebe_73d5_4a13_89d3_e093b53c81d1,"{""user"": ""dm250067""}"
dc1cdd3e-111f-440e-ae0e-c0b3336c0716,training,TDS_MAPPER_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,TDS_STO_TRAINING_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,"{""user"": ""dm250067""}"
57d6b26b-7691-4de9-b465-1d48affe93a8,scoring,TDS_MAPPER_57d6b26b_7691_4de9_b465_1d48affe

### 5.4 - List of Hypermodels ( models and mappers mapping)

In [20]:
sto.list_hyper_models()

CREATION_DATE,ID,ID_MODEL,ID_MAPPER_TRAINING,ID_MAPPER_SCORING,METADATA
2025-01-08 05:26:48.160000-05:,654b028b-741e-4a46-8e5d-d966ac8a07b1,1feb2a86-622c-4c56-8e94-4c367d1a232e,05842693-ddf1-4589-8d7b-a7ea4e952b95,a93604aa-2204-45a6-b4f5-1edab0414da1,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 05:31:04.970000-05:,c5ef0032-dd20-4b48-817e-6fe7e3c42e55,505ea202-bbf4-4b2c-9639-2a18176d97f1,dc1cdd3e-111f-440e-ae0e-c0b3336c0716,431b1e0c-57a4-4ae8-bb3e-3a2372349857,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 06:33:41.570000-05:,f17adbc1-cd51-4281-87ca-2fb179d4ccd0,479b48a6-e0e5-4d18-b637-de6f1ee63d8a,2e06e335-a352-4480-805d-182f66dca5e7,cddb9b47-acc7-4638-a61b-802aea11f0d7,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 05:36:52.330000-05:,91fb8620-0b80-4cec-a8a5-e1de531e608e,06e9bc9e-32f1-4f7b-88f9-5a09e1ac8204,72ef3ebe-73d5-4a13-89d3-e093b53c81d1,d14273b9-df63-444b-98ce-0bad17935b12,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 06:28:41.130000-05:,533fd2f6-cdf9-4c18-bfd1-11f6c017a587,0fe42030-d412-411b-9695-8571c07507a0,fca98df7-3e26-41f5-a8a8-0036c348fb9a,57d6b26b-7691-4de9-b465-1d48affe93a8,"{""user"": ""dm250067"", ""project"": ""test""}"


In [21]:
model.get_model_predictions()

demo_user TDS_SCORES_cddb9b47_acc7_4638_a61b_802aea11f0d7


TD_TIMECODE,ID_PROCESS,ID_TRAINED_MODEL,Partition_ID,ID,Y1_prediction
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,29371,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,21115,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,0ea9f0ea-01ae-47b5-a207-a75487efe313,9,18864,-0.05335791349929137
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,24211,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,0fd9d643-9a31-4560-9ab9-4e8fbc7194cb,6,39884,0.0709809863307359
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,20083,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,27307,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,0ea9f0ea-01ae-47b5-a207-a75487efe313,9,13198,-0.04490876728584044
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,25243,0.018697822801857347
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_cddb9b47_acc7_4638_a61b_802aea11f0d7,f68bdbf5-56ef-4e81-8e35-6e3c0a812b93,4,22147,0.018697822801857347


In [22]:
tdml.remove_context()

True